In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Ielādē datus no katra Excel sheet
elevenify_df = pd.read_excel('/content/drive/MyDrive/bakalauraDarbs/xG_full.xlsx', sheet_name='ElevenifyPrognozes', index_col=0)
my_predictions_df = pd.read_excel('/content/drive/MyDrive/bakalauraDarbs/xG_full.xlsx', sheet_name='AutoraPrognozes', index_col=0)
actual_xg_df = pd.read_excel('/content/drive/MyDrive/bakalauraDarbs/xG_full.xlsx', sheet_name='FaktiskaisXG', index_col=0)
actual_goals_df = pd.read_excel('/content/drive/MyDrive/bakalauraDarbs/xG_full.xlsx', sheet_name='FaktiskieVarti', index_col=0)

# Pārliecinās, ka kolonnu nosaukumi ir string tipa
elevenify_df.columns = elevenify_df.columns.astype(str)
my_predictions_df.columns = my_predictions_df.columns.astype(str)
actual_xg_df.columns = actual_xg_df.columns.astype(str)
actual_goals_df.columns = actual_goals_df.columns.astype(str)

# Inicializē tukšus sarakstus rezultātu uzkrāšanai
rmse_elevenify = []
mae_elevenify = []
rmse_my = []
mae_my = []
pearson_corr = []
rmse_my_vs_goals = []
mae_my_vs_goals = []

# Apstrādā katru spēļu nedēļu
for week in range(1, 36): # Aizvadītas 35 spēļu nedēļas
    week_str = str(week)  # Nedēļas numuru pārvērš par string, jo tādas ir sheet kolonnas

    # Izvelk attiecīgās nedēļas prognozes/faktus no katra DataFrame
    elevenify_week = elevenify_df[week_str]
    my_predictions_week = my_predictions_df[week_str]
    actual_xg_week = actual_xg_df[week_str]
    actual_goals_week = actual_goals_df[week_str]

    # Apvieno datus DataFrame un atstāj tikai pilnos ierakstus (bez NaN)
    week_df = pd.DataFrame({
        'elevenify': elevenify_week,
        'my': my_predictions_week,
        'xg': actual_xg_week,
        'goals': actual_goals_week
    }).dropna()

    # Ja kārtā nav datu, pāriet pie nākamās
    if week_df.empty:
        continue

    # Aprēķina RMSE un MAE metrikas Elevenify prognozēm pret faktiskajiem xG
    rmse_elevenify.append(np.sqrt(mean_squared_error(week_df['xg'], week_df['elevenify'])))
    mae_elevenify.append(mean_absolute_error(week_df['xg'], week_df['elevenify']))

    # Aprēķina RMSE un MAE metrikas autora prognozēm pret xG
    rmse_my.append(np.sqrt(mean_squared_error(week_df['xg'], week_df['my'])))
    mae_my.append(mean_absolute_error(week_df['xg'], week_df['my']))

    # Aprēķina RMSE un MAE metrikas autora prognozēm pret faktisko vārtu skaitu
    rmse_my_vs_goals.append(np.sqrt(mean_squared_error(week_df['goals'], week_df['my'])))
    mae_my_vs_goals.append(mean_absolute_error(week_df['goals'], week_df['my']))

    # Aprēķina Pīrsona korelācijas koeficientu starp faktiskajiem xG un faktiskajiem vārtiem
    corr, _ = pearsonr(week_df['xg'], week_df['goals'])
    pearson_corr.append(corr)

# Aprēķina vidējo vērtību katrai metrikai visām nedēļām
avg_rmse_elevenify = np.mean(rmse_elevenify)
avg_mae_elevenify = np.mean(mae_elevenify)
avg_rmse_my = np.mean(rmse_my)
avg_mae_my = np.mean(mae_my)
avg_pearson_corr = np.mean(pearson_corr)
avg_rmse_my_vs_goals = np.mean(rmse_my_vs_goals)
avg_mae_my_vs_goals = np.mean(mae_my_vs_goals)

# Aprēķina relatīvo atšķirību starp autora un Elevenify RMSE/MAE (procentos)
rel_diff_rmse = (avg_rmse_my - avg_rmse_elevenify) / avg_rmse_elevenify * 100
rel_diff_mae = (avg_mae_my - avg_mae_elevenify) / avg_mae_elevenify * 100

# Izvada rezultātus
print(f"Vidējais Elevenify RMSE vs xG: {avg_rmse_elevenify}")
print(f"Vidējais Elevenify MAE vs xG: {avg_mae_elevenify}")
print(f"Vidējais Autora RMSE vs xG: {avg_rmse_my}")
print(f"Vidējais Autora MAE vs xG: {avg_mae_my}")
print(f"Vidējā Pearsona korelācija starp xG un gūtajiem vārtiem: {avg_pearson_corr}")
print(f"Vidējais Autora RMSE vs Faktiskie vārti: {avg_rmse_my_vs_goals}")
print(f"Vidējais Autora MAE vs Faktiskie vārti: {avg_mae_my_vs_goals}")
print(f"Relatīvā atšķirība RMSE (Autors vs Elevenify, pret xG): {rel_diff_rmse:.2f}%")
print(f"Relatīvā atšķirība MAE (Autors vs Elevenify, pret xG): {rel_diff_mae:.2f}%")

Vidējais Elevenify RMSE vs xG: 0.710273740011883
Vidējais Elevenify MAE vs xG: 0.5615952840387409
Vidējais Autora RMSE vs xG: 0.7660179995668602
Vidējais Autora MAE vs xG: 0.5990914361249834
Vidējā Pearsona korelācija starp xG un gūtajiem vārtiem: 0.5690799413014354
Vidējais Autora RMSE vs Faktiskie vārti: 1.1369758022633076
Vidējais Autora MAE vs Faktiskie vārti: 0.892856730733599
Relatīvā atšķirība RMSE (Autors vs Elevenify, pret xG): 7.85%
Relatīvā atšķirība MAE (Autors vs Elevenify, pret xG): 6.68%
